In [14]:
import datetime as dt
import itertools
from pathlib import Path

import cartopy.crs as ccrs
import dask
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import xarray as xr

from remake import load_remake

from mcs_prime import McsTracks, McsTrack, PATHS


Load all tracks from 2019/01-2020/01

In [2]:
tracks = McsTracks.open(PATHS['statsdir'] / 'mcs_tracks_final_extc_20190101.0000_20200101.0000.nc', 
                                 PATHS['pixeldir'])

In [3]:
tracks.dstracks

<xarray.Dataset>
Dimensions:                  (tracks: 32843, times: 400, mergers: 50, nmaxpf: 3)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 ... 32839 32840 32841 32842
  * times                    (times) int64 0 1 2 3 4 5 ... 395 396 397 398 399
  * mergers                  (mergers) int64 0 1 2 3 4 5 6 ... 44 45 46 47 48 49
  * nmaxpf                   (nmaxpf) int64 0 1 2
Data variables: (12/55)
    track_duration           (tracks) int32 ...
    start_status             (tracks) float64 ...
    end_status               (tracks) float64 ...
    start_basetime           (tracks) datetime64[ns] 2019-01-01T00:30:00 ... ...
    end_basetime             (tracks) datetime64[ns] 2019-01-02T12:30:00 ... ...
    start_split_cloudnumber  (tracks) float64 ...
    ...                       ...
    pf_mcsstatus             (tracks, times) float64 ...
    movement_distance        (tracks, times) float32 ...
    movement_speed           (tracks, times) float32 ...
    movement_theta           (tracks, times) float32 ...
    movement_distance_x      (tracks, times) float32 ...
    movement_distance_y      (tracks, times) float32 ...
Attributes: (12/23)
    Title:                       Statistics of each MCS track
    Institution:                 Pacific Northwest National Laboratory
    Contact:                     Zhe Feng, zhe.feng@pnnl.gov
    Created_on:                  Mon Feb 28 05:50:54 2022
    startdate:                   20190101.0000
    enddate:                     20200101.0000
    ...                          ...
    PF_PF_min_majoraxis_thresh:  20
    coefs_pf_area:               [2874.05    89.825]
    coefs_pf_rr:                 [3.01657   0.0144461]
    coefs_pf_skew:               [0.194462  0.0100072]
    coefs_pf_heavyratio:         [3.419024 0.438709]
    max_speed_thresh:            50

In [13]:
p = np.isnan(tracks.dstracks.start_split_cloudnumber.values).sum() / len(tracks.dstracks.tracks) * 100
print(f'Natural initiation: {p:0.1f}%')
print(f'Merge/split:        {(100 - p):0.1f}%')

Natural initiation: 63.8%
Merge/split:        36.2%


In [15]:
rmk = load_remake('../../remakefiles/mcs_env_cond_figs.py')

In [18]:
t = rmk.PlotCombinedMcsLocalEnvPrecursorMeanValue.tasks[0]

In [19]:
t.inputs

{'mcs_local_env_2001_1': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2001/01/lifecycle_mcs_local_env_2001_01.nc'),
 'mcs_local_env_2001_2': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2001/02/lifecycle_mcs_local_env_2001_02.nc'),
 'mcs_local_env_2001_3': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2001/03/lifecycle_mcs_local_env_2001_03.nc'),
 'mcs_local_env_2001_4': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2001/04/lifecycle_mcs_local_env_2001_04.nc'),
 'mcs_local_env_2001_5': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2001/05/lifecycle_mcs_local_env_2001_05.nc'),
 'mcs_local_env_2001_6': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2001/06/lifecycle_mcs_local_env_2001_06.nc'),
 'mcs_local_env_2001_7': PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_e

In [21]:
inputs_2019 = [p for k, p in t.inputs.items() if '2019' in k]

ds = xr.open_mfdataset(inputs_2019, combine='nested', concat_dim='tracks')
ds['tracks'] = np.arange(0, ds.dims['tracks'], 1, dtype=int)

In [23]:
ds

<xarray.Dataset>
Dimensions:                                             (tracks: 32843,
                                                         radius: 5, times: 424,
                                                         percentile: 5)
Coordinates:
  * tracks                                              (tracks) int64 0 ... ...
  * radius                                              (radius) int64 1 ... ...
  * times                                               (times) int64 -24 ......
  * percentile                                          (percentile) int64 10...
Data variables: (12/24)
    mean_cape                                           (tracks, radius, times) float64 dask.array<chunksize=(2673, 5, 424), meta=np.ndarray>
    percentile_cape                                     (tracks, radius, percentile, times) float64 dask.array<chunksize=(2673, 5, 5, 424), meta=np.ndarray>
    mean_tcwv                                           (tracks, radius, times) float64 dask.array<chunksize=(2673, 5, 424), meta=np.ndarray>
    percentile_tcwv                                     (tracks, radius, percentile, times) float64 dask.array<chunksize=(2673, 5, 5, 424), meta=np.ndarray>
    mean_shear_0                                        (tracks, radius, times) float64 dask.array<chunksize=(2673, 5, 424), meta=np.ndarray>
    percentile_shear_0                                  (tracks, radius, percentile, times) float64 dask.array<chunksize=(2673, 5, 5, 424), meta=np.ndarray>
    ...                                                  ...
    mean_theta_e_mid                                    (tracks, radius, times) float64 dask.array<chunksize=(2673, 5, 424), meta=np.ndarray>
    percentile_theta_e_mid                              (tracks, radius, percentile, times) float64 dask.array<chunksize=(2673, 5, 5, 424), meta=np.ndarray>
    mean_delta_3h_cape                                  (tracks, radius, times) float64 dask.array<chunksize=(2673, 5, 424), meta=np.ndarray>
    percentile_delta_3h_cape                            (tracks, radius, percentile, times) float64 dask.array<chunksize=(2673, 5, 5, 424), meta=np.ndarray>
    mean_delta_3h_tcwv                                  (tracks, radius, times) float64 dask.array<chunksize=(2673, 5, 424), meta=np.ndarray>
    percentile_delta_3h_tcwv                            (tracks, radius, percentile, times) float64 dask.array<chunksize=(2673, 5, 5, 424), meta=np.ndarray>
Attributes: (12/18)
    created by:                      /home/users/mmuetz/projects/mcs_prime/ct...
    calling file source:             """Remakefile to calculate local environ...
    project repository:              https://github.com/markmuetz/MCS_PRIME
    remake version:                  ['0', '6', '3', '0', 'beta']
    remake repository:               https://github.com/markmuetz/remake
    task:                            fd5d7eb8c4 LifecycleMcsLocalEnv(year=201...
    ...                              ...
    production_mcs_prime_git_info:   GitInfo(loc='/home/users/mmuetz/projects...
    production_remake_git_info:      GitInfo(loc='/home/users/mmuetz/projects...
    production_conda_env_details:    name: mcs_prime_env\nchannels:\n  - cond...
    production_machine_lspcu:        Architecture:          x86_64\nCPU op-mo...
    production_machine_lsb_release:  Distributor ID:\tCentOS\nDescription:\tC...
    production_machine_uname:        Linux host613.jc.rl.ac.uk 3.10.0-1160.92...

In [26]:
ds_natural = ds.isel(tracks=np.isnan(tracks.dstracks.start_split_cloudnumber.values))